In [1]:
import json
import urllib2
import urllib
import xmltodict
import os
import string
from ebooklib import epub 
from  comFunc import *
from multiprocessing import Pool
import time
import re
import functools


In [112]:

def movet(titles):
    #print titles
    series=""
    index=-1
    metakey='http://purl.org/dc/elements/1.1/'
    f=titles[1][0]+"/"+titles[0]+".epub"
    ti=' '.join(titles[1][1][u'title'].encode('ascii','ignore').replace("&"," and ").split())
    seri=""
    if ti.count('(')>0:
        
        #print seri
        seri=ti[ti.index('(')+1:ti.index(")")].split()
        ti=ti[:ti.index('(')]+ti[ti.index(")")+1:]
        try:
            index=int(''.join(s for s in seri[-1] if s in string.digits))
        except:
            index=-1
        series=' '.join(s for s in seri[:-1] if s.lower()!="book").strip()
        
        if index<0:
            series=' '.join(s for s in seri if s.lower()!="book").strip()

    series=''.join(s for s in series if s in string.letters+" ")
    if len(titles[1][1][u'authors'])>0:
        auth=' '.join(titles[1][1][u'authors'][0].encode('ascii','ignore').split())
    else:
        auth='Unknown'
    
    #remove extras
    auth=''.join(i for i in auth if i in string.letters+" .-")
    ti=''.join(i for i in ti if i in string.letters+string.digits+" .-_,#")
    if u'industryIdentifiers' in titles[1][1].keys() and len(titles[1][1][u'industryIdentifiers'])>0:
        isbn=titles[1][1][u'industryIdentifiers'][0][u'identifier']
        for idf in titles[1][1][u'industryIdentifiers']:
            if idf[u'type']==u"ISBN_13":
                isbn=idf[u'identifier'].encode('ascii','ignore')
    else:
        isbn="UN"
    if len(ti.split())>6:
        ti=' '.join(ti.split()[:4])
    
    name=auth+" - "+ti
    if len(series)>1 and index >0:
        if len(series.split())>4:
            ss=' '.join(series.split()[:3])
        else:
            ss=series
        name=name+" - "+ss+" #"+str(index)
    tmp="/tmp/"+name.strip()+".epub"
    tt="/media/sf_lbuntu/books/"+name.strip()+".epub"
    if not os.path.exists(f):
        return "NE",f,tt
    if os.path.exists(tt):
            return "E",f,tt
    else:
        try:
            book=epub.read_epub(f)

        except:
            return "R",f,tt
        book.metadata={}

    #if 'creator' in book.metadata[metakey].keys():
    #book.set_unique_metadata( namespace, name, value, others=None)
    #add_metadata(self, namespace, name, value, others=None):
        if u'title' in titles[1][1].keys():
            book.set_title(titles[1][1][u'title'])
        if u'authors' in titles[1][1].keys():
            #book.metadata[metakey].pop('creator')
            for ar in titles[1][1][u'authors']:
                book.add_author(ar.encode('ascii','ignore'))

    #if 'identifier' in book.metadata[metakey].keys():
        #book.metadata[metakey].pop('identifier')
        if isbn!="UN":
            book.set_identifier(isbn)
        for md in titles[1][1].keys():
            if md not in [u'authors',u'title']:
                if type(titles[1][1][md])==unicode:
                    #book.add_metadata( "DC", md.encode('ascii','ignore'), titles[1][1][md].encode('ascii','ignore'), others=None)
                    book.add_metadata( "DC", "calibre:"+md.encode('ascii','ignore'), titles[1][1][md].encode('ascii','ignore'), others=None)
                elif type(titles[1][1][md])==list:
                    for mddd in titles[1][1][md]:
                        mdd=mddd
                        if type(mdd)==unicode:
                            #book.add_metadata( "DC", mdd.encode('ascii','ignore'), mdd.encode('ascii','ignore'), others=None)
                            book.add_metadata( "DC", "calibre:"+md.encode('ascii','ignore'), mdd.encode('ascii','ignore'), others=None)
        #print book.metadata[metakey].keys(


        #print titles[1][1].keys()
        #book.metadata[metakey]=titles[1][1]

        try:
            epub.write_epub(tmp,book,{} )
        except:
            return "W",f,tt
            

        if os.path.exists(tmp):
            
            if len(series)>0 and "series" not in book.metadata[metakey].keys(): 
                #print series,seri,index,ti
                if index<0:
                    index=1
                !ebook-meta "$tmp" --series "$series" --index "$index" >>/dev/null 2>>/dev/null
            s=!mv "$tmp" "$tt"
            #!ebook-meta "$tmp" --title "$ti" --authors "$auth" --language "eng" >>/dev/null 2>>/dev/null
            #print s
            #if index>0 and len(series)>0: 
             #   !ebook-meta "$tmp" --series "$series" --index "$index" >>/dev/null 2>>/dev/null
            if len(s)==0:
                return "S",f,tt
            else:
                return "M",f,tt
        else:
            return "F",f,tt

#E=Exists, F=failed, S=success, W=Writing error, R=reading error, A=Author problem

In [113]:
#save_obj(titles,'data/titlesGA')
titles=load_obj('data/titlesGA')#{}
#unpacktitles
uptit=[[k,titles[k]] for k in titles.keys()]
prockeys=[]#load_obj('data/prockeysIm')
flist=list(filter(lambda x: x[0] not in prockeys and type(x[1][1])==dict, uptit))

len(flist),len(titles.keys()),len(prockeys)

(495, 495, 0)

In [108]:
movet(flist[0])

('E',
 '/media/some/sophie-hhd/transmitt/fic/Fictional Literature/New York Times Bestsellers-February 25 2013(Epub, Mobi)/17 Tenth of December - George Saunders/George Saunders - Tenth of December_ Stories.epub',
 '/media/sf_lbuntu/books/George Saunders - Tenth of December.epub')

In [114]:
pbooks=[]#load_obj('data/procbooksIm')
t0 = time.time()

p = Pool(10)
books=p.map(movet, flist)
#p.join()
p.terminate()
p.close()

save_obj(prockeys,'data/prockeysIm')
save_obj(pbooks,'data/procbooksIm')

t1=time.time()
total = t1-t0
#
#
print('time: ',total)

('time: ', 0.36464500427246094)


In [110]:
print len(books),len(prockeys)

495 0


In [116]:
print " R: ",len(list (filter(lambda x:x[0]  =="R",books)))
print " W: ",len(list (filter(lambda x:x[0]  =="W",books)))
print " S: ",len(list (filter(lambda x:x[0]  =="S",books)))
print " F: ",len(list (filter(lambda x:x[0]  =="F",books)))
print " E: ",len(list (filter(lambda x:x[0]  =="E",books)))
print " A: ",len(list (filter(lambda x:x[0]  =="A",books)))
print "NE: ",len(list (filter(lambda x:x[0]  =="NE",books)))
print " M: ",len(list (filter(lambda x:x[0]  =="M",books)))

 R:  0
 W:  0
 S:  0
 F:  0
 E:  0
 A:  0
NE:  495
 M:  0


In [78]:
res={}
res["R"] = [x[1:] for x in list (filter(lambda x:x[0]  =="R",books))]
res["W"] = [x[1:] for x in list (filter(lambda x:x[0]  =="W",books))]
res["S"] = [x[1:] for x in list (filter(lambda x:x[0]  =="S",books))]
res["F"] = [x[1:] for x in list (filter(lambda x:x[0]  =="F",books))]
res["E"] = [x[1:] for x in list (filter(lambda x:x[0]  =="E",books))]
res["A"] = [x[1:] for x in list (filter(lambda x:x[0]  =="A",books))]
res["M"] = [x[1:] for x in list (filter(lambda x:x[0]  =="M",books))]

In [105]:
procN=[]
for r in sorted(res['E']):
    org=r[0][r[0].rindex('/')+1:r[0].rindex('.')]
    org=" - ".join(reversed(org.split(' - ')))
    nw=r[1][r[1].rindex('/')+1:r[1].rindex('.')]
    sm=similarity(org,nw)
    if sm>.6:
        f=r[0]
        if os.path.exists(f):
            print f
    #
    #print '{0:60s} {1:50s} {2:.1f}'.format(org,nw,sm)

In [105]:
def fconv(f):
    #f=re.sub(" \(v[0-9].[0-9]\)","",f)
    
    f=f.encode('ascii','ignore')
    
    namer=f[f.rindex("/")+1:f.rindex('.')]
    name=''.join(s for s in namer if s in string.letters)
    t="/tmp/conv"+name+".txt"
    im="/tmp/conv"+name+".jpg"
    tmp="/tmp/conv"+name+".epub"
    ff= f[:f.rindex('(')-1]+" (v5.0) (epub).epub"
    !ebook-meta "$ff" --get-cover "$im" >>/dev/null 2>>/dev/null
    !ebook-convert "$ff" "$t" >>/dev/null 2>>/dev/null
    !ebook-convert "$t" "$tmp" --cover "$im" 2>>/dev/null >>/dev/null 

    v=movet(["conv"+name,
           ("/tmp",titles[namer][1],titles[namer][2])])
    !rm "$t" "$tmp" "$im" 2>>/dev/null >>/dev/null 
    
    
    return v

In [72]:
titles['Nelson Johnson - Boardwalk Empire_ The Birth, H_ity'][0]

'/media/some/sophie-hhd/transmitt/fic/Fictional Literature/True Crime Collection/Boardwalk Empire_ The Birth, High Times, and Corruption of Atlantic City - Nelson Johnson'

In [73]:
t0 = time.time()
p=Pool(10)
booksR=p.map(fconv,res['R'])
p.terminate()
p.close
t1=time.time()
total = t1-t0
print('time: ',total)

('time: ', 8.06489896774292)


In [67]:
print "R: ",len(list (filter(lambda x:x[0]  =="R",booksR)))
print "W: ",len(list (filter(lambda x:x[0]  =="W",booksR)))
print "S: ",len(list (filter(lambda x:x[0]  =="S",booksR)))
print "F: ",len(list (filter(lambda x:x[0]  =="F",booksR)))
print "E: ",len(list (filter(lambda x:x[0]  =="E",booksR)))
print "A: ",len(list (filter(lambda x:x[0]  =="A",booksR)))

print "M: ",len(list (filter(lambda x:x[0]  =="M",booksR)))

R:  6
W:  0
S:  7
F:  0
E:  0
A:  0
M:  0


In [68]:
resr={}
resr["R"] = [x[1] for x in list (filter(lambda x:x[0]  =="R",booksR))]
resr["W"] = [x[1] for x in list (filter(lambda x:x[0]  =="W",booksR))]
resr["S"] = [x[1] for x in list (filter(lambda x:x[0]  =="S",booksR))]
resr["F"] = [x[1] for x in list (filter(lambda x:x[0]  =="F",booksR))]
resr["E"] = [x[1] for x in list (filter(lambda x:x[0]  =="E",booksR))]
resr["A"] = [x[1] for x in list (filter(lambda x:x[0]  =="A",booksR))]
resr["M"] = [x[1] for x in list (filter(lambda x:x[0]  =="M",booksR))]

In [ ]:
t

In [ ]:
cnt=0
for f in flist:
    print cnt,f[0]
    cnt+=1